# State College/Penn State
**Climatology:** State College/Penn State has a humid continental climate, with distinct seasons including a hot summer and a cold winter. The surrounding mountains is influencing the region with stronger winters and consistent summers. Average precipitation approximately 2 to 4 inches with ample snowfall.\
**Ecology:** The surrounding of Penn State has diverse vegetation including hardwood forests and meadows with a variety of wildlife. Penn State is known for its agriculture programs which have influenced the local ecology with its environmental and animal management practices.\
**Communities and Infrastructure:** State College/Penn State is a well-developed college town with the university and the Downtown State College along with solid homes, restaurants, roads, etc.

# Penn State's Hurricane Ivan in 2004
According to Penn Live, the remnants of Hurricane Ivan dumped from 4 to 7 inches of rain in central Pennsylvania September 18-19, 2004. The excessive rain from the hurricane caused flash floods as evidented by these images:

![State College’s Memorial Field filled with water on Friday. Kathy Derge Gipe Photo provided to Centre Daily](https://www.centredaily.com/latest-news/il82qf/picture216111125/alternates/FREE_960/Gipe.jpg)\
**State College’s Memorial Field filled with water on Friday. Kathy Derge Gipe Photo provided to Centre Daily**

![The Cumberland Valley football team lines up to play defense halfway in water against the State College Area High School offense at Memorial field in State College, Pa., Friday, Sept. 17, 2004. Rain from Hurricane Ivan steadily flooded the field throughout the first half which was postponed at halftime. (AP Photo/Centre Daily Times, Nabil K. Mark)](http://image.pennlive.com/home/penn-media/width960/img/patriot-news/photo/2015/09/18/-b01cc37344cb598a.jpg)\
**The Cumberland Valley football team lines up to play defense halfway in water against the State College Area High School offense at Memorial field in State College, Pa., Friday, Sept. 17, 2004. Rain from Hurricane Ivan steadily flooded the field throughout the first half which was postponed at halftime. (AP Photo/Centre Daily Times, Nabil K. Mark)**

In [1]:
import subprocess
from io import BytesIO

import folium
import hvplot.pandas
import pandas as pd
import requests
import warnings


warnings.filterwarnings('ignore')

In [2]:
# Find a location through National Water Information System Mapper
# URL: https://dashboard.waterdata.usgs.gov/app/nwd/en
# Click into the monitoring station of interest
# Nav Bar: Site Page, Data

# Site Page: 
# https://waterdata.usgs.gov/monitoring-location/01546400/#parameterCode=00065&period=P7D&showMedian=true

sg_lat = 40.8336726
sg_lon = -77.82750108

popupdata = {
    "Nickname": "Stream Gauge near Penn State",
    "Site Name": "Spring Creek at Houserville, PA",
    "Site ID (Type)": "USGS 01546400 (Surface Water, Stream)"
}
popup_df = pd.DataFrame([popupdata])

# Initialize map and tweak settings
m = folium.Map(
    # Location to display
    location=(sg_lat, sg_lon),
    # Turns off annoying zooming while trying to scroll to the next cell
    scrollWheelZoom=False,
    tiles="Stamen Toner",
    width=1000,
    height=800,
    zoom_start=13)

html_table = popup_df.to_html(classes="table table-striped table-hover", index=False)
popup = folium.Popup(html_table, max_width=400)

# Put a marker at the stream gauge location
folium.Marker([sg_lat, sg_lon], 
              popup=popup
            ).add_to(m)

# Display the map
m

In [3]:
# Data: 
# https://waterdata.usgs.gov/nwis/inventory?site_no=01546400
# Select Daily Data
# Output format: TSV

usgs_nwis_url = 'https://waterdata.usgs.gov'\
                '/nwis/dv?cb_00060=on&format=rdb&'\
                'site_no=01546400&legacy=&referred_module=sw'\
                '&period=&begin_date=1985-10-01&end_date=2023-09-19'
water_data = requests.get(usgs_nwis_url)

In [4]:
# Refer to the following for help:
# https://waterdata.usgs.gov/nwis/?tab_delimited_format_info
# https://help.waterdata.usgs.gov/faq/about-tab-delimited-output
discharge_df = pd.read_csv(
    BytesIO(water_data.content),
    comment='#',
    skiprows=[29],
    delimiter='\t',
    index_col='datetime',
    parse_dates=['datetime']
)
discharge_df = discharge_df.rename(columns={'118932_00060_00003':
                            'discharge_cubic_ft_per_second_mean'})

# 1985 October - 2023 September Time Series Water Analysis
The following discharge plot shows daily streamflow average near Penn State from 1985 to 2023. There are several spikes, notably 1996, 2004, and 2008, which correlates with these hurricanes: Tropical Storm Fran (September 1996), Hurricane Ivan (September 2004), and Tropical Storm Hanna (September 2008).

In [5]:
discharge_plot = discharge_df.hvplot(x='datetime', y='discharge_cubic_ft_per_second_mean',
                                    title='Daily Streamflow Average (Near Penn State)',
                                    xlabel='Date', ylabel='Discharge (ft^3 per second)',
                                    width=1000)
discharge_plot

:Curve   [datetime]   (discharge_cubic_ft_per_second_mean)

In [6]:
yearly_df = discharge_df[['discharge_cubic_ft_per_second_mean']]
yearly_df = yearly_df.resample('AS').max()

Maximum annual stream discharge (produced from daily average) showing 2004 being the worst year with most impacted water in recent times.

In [7]:
annual_plot = yearly_df.hvplot(x='datetime', y='discharge_cubic_ft_per_second_mean',
                                    title='Streamflow Annual Maxima (Near Penn State)',
                                    xlabel='Year', ylabel='Discharge (ft^3 per second)',
                                    width=1000)
annual_plot

:Curve   [datetime]   (discharge_cubic_ft_per_second_mean)

# The flood return periods are calculated for the period 1985-2022 as follows:

In [8]:
return_period_df = yearly_df['1985':'2022'].copy()
return_period_df['exceedance_probability'] = (
    return_period_df.rank(ascending=False) / len(return_period_df)
)
return_period_df['return_period'] = 1 / return_period_df.exceedance_probability
return_period_df = return_period_df.rename(columns={'discharge_cubic_ft_per_second_mean': 'discharge_cftps'})
return_period_df

,discharge_cftps,exceedance_probability,return_period
datetime,,,
1985-01-01,191.0,0.947368,1.055556
1986-01-01,596.0,0.289474,3.454545
1987-01-01,352.0,0.684211,1.461538
1988-01-01,243.0,0.815789,1.225806
1989-01-01,403.0,0.578947,1.727273
1990-01-01,291.0,0.763158,1.310345
1991-01-01,219.0,0.868421,1.151515
1992-01-01,232.0,0.842105,1.187500
1993-01-01,771.0,0.105263,9.500000


# Without assuming a normal distribution, based on data spanning from 1985 to 2022, 2004 Hurricane Ivan is a 38-year flood event for the Penn State/State College region.

In [9]:
return_period_plot = return_period_df.return_period.hvplot(
                                    title='Flood Return Period (Near Penn State)',
                                    xlabel='Year', ylabel='Return Period',
                                    width=1000)
return_period_plot

:Curve   [datetime]   (return_period)

# Data Source
https://waterdata.usgs.gov/nwis/inventory?site_no=01546400 (Daily Data)

# Citation 
U.S. Geological Survey, 2023, National Water Information System data available on the World Wide Web (USGS Water Data for the Nation), accessed [September 19, 2023], at URL [http://waterdata.usgs.gov/nwis/].

In [10]:
%%capture
%%bash
jupyter nbconvert usgs-nwis-time-series-flood-analysis.ipynb --to html --no-input